In [8]:
import h5py
import numpy as np
from skimage import io, transform
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import sys,os
sys.path.append(os.path.abspath('../'))
import utils.Dataset as u
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as fc
import torch.nn.modules.normalization as nm
import torch.optim as optim
import CNN as cnn

In [9]:
a= 5
torch.tensor(a)

tensor(5)

In [27]:
directory = "/home/chekirou/Documents/SOLI/SoliData/dsp"
# Datasets
train, test = u.split(directory,frames = True, percentage = 0.8, use= 0.001) # returns a split of the frames 80% train, 20% test, for 80% of the data

t = transforms.Compose([u.Reshape(), u.Rescale((224,224)), u.ToTensor()])
training_set = u.Data(directory,train,transform = t) # open the dataset"""

In [28]:
train, test = u.split(directory,frames = True, percentage = 0.8, use= 0.2) # returns a split of the frames 80% train, 20% test, for 80% of the data

# Generators
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
t = transforms.Compose([u.Reshape(), u.Rescale((224,224)), u.ToTensor()]) # composition of transformations
training_set = u.Data(directory,train,transform = t) # open the dataset
training_generator = DataLoader(training_set, **params)

In [ ]:
for i, (s, l) in enumerate(training_generator):
    print(i, j,l)

In [31]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True # ca marche pas chez moi,peut etre qu'il faut que j'installe un truc
# Parameters, num_workers c'est les threads
"""device = torch.device("cpu")"""
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

max_epochs = 1 # j'ai mis a 1 pour le test pour aller vite
directory = "/home/celina/data/dsp"
# Datasets
train, test = u.split(directory,frames = True, percentage = 0.8, use= 0.2) # returns a split of the frames 80% train, 20% test, for 80% of the data

# Generators

t = transforms.Compose([u.Reshape(), u.Rescale((224,224)), u.ToTensor()]) # composition of transformations
training_set = u.Data(directory,train,transform = t) # open the dataset
training_generator = DataLoader(training_set, **params)


validation_set = u.Data(directory,test,transform = t) # open the dataset
validation_generator = DataLoader(validation_set, **{'batch_size': 1,'shuffle': False,'num_workers': 6})

model = cnn.Net().double().to(device)
optimizer =  optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
# Train the model


loss_list = []
acc_list = []
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(training_generator):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        labels = labels.view(images.shape[0]).long()
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100  == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
            running_loss = 0.0
        

[1,   100] loss: 237.007
[1,   200] loss: 225.904
[1,   300] loss: 228.458
[1,   400] loss: 224.241
[1,   500] loss: 220.885
[1,   600] loss: 220.955
[1,   700] loss: 220.259
[1,   800] loss: 218.642


In [32]:
training_set2 = u.Data(directory,train[:200],transform = t)

In [33]:
training_generator2 = DataLoader(training_set2, **{'batch_size': 1,'shuffle': False,'num_workers': 6})

In [32]:
acc = 0
for images, labels in (validation_generator):
    images, labels = images.to(device), labels.long().to(device)
    outputs = model(images)
    prob_dist = torch.distributions.Categorical(outputs) # probs should be of size batch x classes
    predictions = prob_dist.sample().to(device)
    if predictions == labels[0]:
        acc += 1
    
accuracy = acc / len(validation_generator)

In [33]:
accuracy

0.4407684098185699

In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [6]:
torch.cuda.is_available()

True

In [19]:
torch.cuda.empty_cache()

In [8]:
probs = torch.Tensor([ [0.1, 0.2, 0.7], [0.1, 0.8, 0.1], [0.4, 0.5, 0.1] ])
prob_dist = torch.distributions.Categorical(probs) # probs should be of size batch x classes
a = prob_dist.sample()

In [9]:
a

tensor([1, 1, 0])

In [35]:
torch.save(model, "/home/celina/DeepSoli/models/model_version0.1_AdamOpt")

In [36]:
# model = torch.load("/home/celina/DeepSoli/models/model_version0.1_AdamOpt")

In [7]:
[1,2,3][:4]

[1, 2, 3]